# dev001
map@12の実装

In [1]:
import os
import sys
from dotenv import load_dotenv
load_dotenv
sys.path.append(os.getenv('UTILS_PATH'))

import pandas as pd
import numpy as np

In [2]:
INPUT_DIR = os.getenv('INPUT_DIR')
OUTPUT_DIR = os.getenv('OUTPUT_DIR')

In [3]:
transactions = pd.read_csv(INPUT_DIR + 'transactions_train.csv', dtype={'article_id':'str'}, parse_dates=['t_dat'])
sample = pd.read_csv(INPUT_DIR + 'sample_submission.csv')

In [4]:
val_start_date = '2020-09-16'
train = transactions[transactions['t_dat']<val_start_date].copy()
valid = transactions[transactions['t_dat']>=val_start_date].copy()

# validの正解データ作成

In [5]:
valid_unique = valid[['customer_id', 'article_id']].drop_duplicates()
valid_agg = valid_unique.groupby('customer_id')['article_id'].apply(list).reset_index()
valid_true = valid_agg['article_id'].tolist()

# 予測値作成
trainの最頻値で予測値を作る

In [6]:
# customerごとの購入数量
customer_agg = train.groupby(['customer_id', 'article_id'])['t_dat'].count().reset_index()
customer_agg = customer_agg.rename(columns={'t_dat':'cnt'})
customer_agg = customer_agg.sort_values(['customer_id', 'cnt'], ascending=False)
customer_agg = customer_agg.groupby('customer_id').head(12)

In [7]:
customer_agg['customer_id'].value_counts()[:10]

800d408d1d6fa043346f5bc5b890e15403a7604406e83cf2ffd89fe564dc4c75    12
7868619814187e50f63413fcf5e878caa5d84218d980117e9a8502e4514062ae    12
786af723bd8a4a25a394c3587a9de7298ddaa1e0a296994c2a1406f424a749ce    12
786aed6081b30249e6f6b720f2a33b4c72a81fea5df4d73ff4a949b89a759167    12
786ad205e74668b7b820e433b51b77235c04bb756bf6046c6b59839f29fd870c    12
786aa101fee83ea1ce2c1259e2963b934088da91d95dc17d1d7831947220452b    12
786a866632896caf719fe9a28fc27c28e59b2aab1909e927e8360c8f13e1b0d0    12
786a7635baf78ef4fa3f8a1c80db94302e91bcc2ee57113f037e6b92dee73231    12
786a4a8208e688c83d4ad11cfa15950d49388882be3efee1ec5f09ee8a730fd9    12
786a3966c222fa15a78c7dea50e68e4a9550fb6b7af76aa9845c72e4d2f0bf90    12
Name: customer_id, dtype: int64

In [8]:
customer_mode = customer_agg.groupby('customer_id')['article_id'].apply(list).reset_index()
customer_mode.head()

,customer_id,article_id
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,"[0568601006, 0797065001, 0176209023, 056860104..."
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,"[0811835004, 0351484002, 0689898002, 072352900..."
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,"[0351484002, 0663713001, 0750424014, 087030400..."
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,"[0732413001, 0742079001]"
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,"[0399061015, 0589440005, 0634249005, 067704900..."


In [9]:
# 全体の購入数量
total_agg = train.groupby('article_id')['t_dat'].count().reset_index()
total_agg = total_agg.rename(columns={'t_dat':'cnt'})
total_agg = total_agg.sort_values(['cnt'], ascending=False)
total_agg = total_agg.head(12)
total_agg.head()

,article_id,cnt
53792,0706016001,49958
53793,0706016002,34802
1707,0372860001,31482
24782,0610776002,30003
70054,0759871002,26309


In [10]:
total_mode = list(total_agg['article_id'].values)
total_mode

['0706016001',
 '0706016002',
 '0372860001',
 '0610776002',
 '0759871002',
 '0464297007',
 '0372860002',
 '0610776001',
 '0399223001',
 '0720125001',
 '0706016003',
 '0156231001']

In [11]:
# 学習期間で購入実績のある顧客はその実績データの最頻値を当てる。ない場合は全体の最頻値を当てる
valid_mode = valid_agg['customer_id'].to_frame().copy()
valid_mode = valid_mode.merge(customer_mode, on='customer_id', how='left')

# listの代入がfillnaやlocではうまく行かないのでforループでatで入れてく（改善の余地あり）
for idx in valid_mode[valid_mode['article_id'].isnull()].index:
    valid_mode.at[idx, 'article_id'] = total_mode

# 購入実績の商品個数が12に満たないものは全体の最頻値も足して12に合わせる
for idx in valid_mode[valid_mode['article_id'].apply(len) < 12].index:
    new_vals = valid_mode.at[idx, 'article_id'] + total_mode
    new_vals = sorted(set(new_vals), key=new_vals.index)
    valid_mode.at[idx, 'article_id'] = new_vals[:12]

valid_pred = valid_mode['article_id'].tolist()
valid_pred[:5]

[['0564358030',
  '0671607001',
  '0704029002',
  '0771602001',
  '0436261001',
  '0482660018',
  '0504155012',
  '0504155018',
  '0537895003',
  '0564358024',
  '0568601007',
  '0620081006'],
 ['0873217004',
  '0880017001',
  '0377277001',
  '0621381012',
  '0640021012',
  '0733749001',
  '0784053005',
  '0823025001',
  '0870962001',
  '0875329001',
  '0889036004',
  '0891322006'],
 ['0872453001',
  '0372860068',
  '0399136061',
  '0556255001',
  '0576897001',
  '0642016001',
  '0659832014',
  '0700926002',
  '0706016001',
  '0706016002',
  '0732842014',
  '0732842015'],
 ['0706016001',
  '0706016002',
  '0372860001',
  '0610776002',
  '0759871002',
  '0464297007',
  '0372860002',
  '0610776001',
  '0399223001',
  '0720125001',
  '0706016003',
  '0156231001'],
 ['0806388001',
  '0569996001',
  '0829017008',
  '0448509014',
  '0456163060',
  '0529953001',
  '0589599001',
  '0600886001',
  '0662980003',
  '0676387001',
  '0685687003',
  '0696797001']]

# MAP@12の計算
「kaggleで勝つデータ分析の技術」より

In [12]:
K = 12
y_i_true = valid_true[0]
y_i_pred = valid_pred[0]

# 予測値の数と重複の確認
assert(len(y_i_pred) <= K)
assert(len(np.unique(y_i_pred)) == len(y_i_pred))

sum_precision = 0.0
num_hits = 0.0

for i, p in enumerate(y_i_pred):
    if p in y_i_true:
        num_hits += 1
        precision = num_hits / (i+1)
        sum_precision += precision
apk = sum_precision / min(len(y_i_true), K)
apk

0.0

In [13]:
def apk(y_true, y_pred, K=12):
    assert(len(y_true) == len(y_pred))
    apks = []
    for idx in range(len(y_true)):
        y_i_true = valid_true[idx]
        y_i_pred = valid_pred[idx]

        # 予測値の数と重複の確認
        assert(len(y_i_pred) <= K)
        assert(len(np.unique(y_i_pred)) == len(y_i_pred))

        sum_precision = 0.0
        num_hits = 0.0

        for i, p in enumerate(y_i_pred):
            if p in y_i_true:
                num_hits += 1
                precision = num_hits / (i+1)
                sum_precision += precision
        apk = sum_precision / min(len(y_i_true), K)
        apks.append(apk)
    return apks


In [14]:
apk(valid_true, valid_pred)[:5]

[0.0, 0.0, 0.0, 0.0, 0.020833333333333332]

In [15]:
# MAP@12
mapa12 = np.mean(apk(valid_true, valid_pred))
print(f'MAP@12 : ' + '{:.5f}'.format(mapa12))

MAP@12 : 0.00904


# sub形式への整形

In [16]:
sample.head()

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0706016001 0706016002 0372860001 0610776002 07...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0706016001 0706016002 0372860001 0610776002 07...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0706016001 0706016002 0372860001 0610776002 07...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0706016001 0706016002 0372860001 0610776002 07...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0706016001 0706016002 0372860001 0610776002 07...


In [17]:
pred = sample['customer_id'].to_frame().copy()
pred = pred.merge(customer_mode, on='customer_id', how='left')

# listの代入がfillnaやlocではうまく行かないのでforループでatで入れてく（改善の余地あり）
for idx in pred[pred['article_id'].isnull()].index:
    pred.at[idx, 'article_id'] = total_mode

# 購入実績の商品個数が12に満たないものは全体の最頻値も足して12に合わせる
for idx in pred[pred['article_id'].apply(len) < 12].index:
    new_vals = pred.at[idx, 'article_id'] + total_mode
    new_vals = sorted(set(new_vals), key=new_vals.index)
    pred.at[idx, 'article_id'] = new_vals[:12]

pred.head()

,customer_id,article_id
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,"[0568601006, 0797065001, 0176209023, 056860104..."
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,"[0811835004, 0351484002, 0689898002, 072352900..."
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,"[0351484002, 0663713001, 0750424014, 087030400..."
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,"[0732413001, 0742079001, 0706016001, 070601600..."
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,"[0399061015, 0589440005, 0634249005, 067704900..."


In [18]:
sub = sample.copy() 
sub['prediction'] = pred['article_id'].apply(lambda x: ' '.join(x))
sub.head()

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601006 0797065001 0176209023 0568601043 06...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0811835004 0351484002 0689898002 0723529001 05...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0351484002 0663713001 0750424014 0870304002 05...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0732413001 0742079001 0706016001 0706016002 03...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0399061015 0589440005 0634249005 0677049001 06...
